# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [109]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [110]:
#read in our csv file
cities_path = "cities.csv"
cities_df = pd.read_csv(cities_path)
#drop index from our csv since we wont need it
cities_df = cities_df.drop(columns=['Unnamed: 0'])
cities_df.head()

,City,Lat,Lng,Temp,Max Temp,Humidity,Cloudiness,Wind Speed,Country
0,Bluff,-46.6000,168.3333,54.00,54.00,90,72,3.00,NZ
1,Lagan,45.3929,47.3707,55.47,55.47,69,59,0.98,RU
2,Esperance,-33.8667,121.9000,50.00,50.00,87,0,6.91,AU
3,Hasaki,35.7333,140.8333,49.14,51.80,43,20,6.91,JP
4,Port Alfred,-33.5906,26.8910,75.20,75.20,37,0,6.98,ZA


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [111]:
#configure gmaps using data from our data frame
gmaps.configure(api_key=g_key)
locations = cities_df[["Lat", "Lng"]].astype(float)
weight= cities_df["Humidity"]

In [112]:
#create figure and add heat layer
fig = gmaps.figure(center=(17, 8), zoom_level=1.50)

heat_layer = gmaps.heatmap_layer(locations, weights=weight, 
                                  dissipating=False, 
                                  max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [113]:
#use .loc to narrow down the results in our data frame. use .drop.na to drop null values
narrow_df = cities_df.loc[(cities_df["Wind Speed"] <= 10) & (cities_df["Cloudiness"]== 0) &
                           (cities_df["Max Temp"] >= 70) & (cities_df["Max Temp"] <= 80)].dropna()
narrow_df

,City,Lat,Lng,Temp,Max Temp,Humidity,Cloudiness,Wind Speed,Country
4,Port Alfred,-33.5906,26.8910,75.20,75.20,37,0,6.98,ZA
85,East London,-33.0153,27.9116,74.64,74.64,34,0,5.93,ZA
106,Safut,32.0336,35.8332,76.15,77.00,33,0,6.98,JO
139,Pisco,-13.7000,-76.2167,71.94,73.40,68,0,5.75,PE
169,Mar del Plata,-38.0023,-57.5575,72.41,73.40,68,0,6.91,AR
199,Lima,-12.0432,-77.0282,72.59,73.99,73,0,6.91,PE
304,Karratha,-20.7377,116.8463,78.12,78.12,68,0,5.48,AU
326,Alexandria,31.2156,29.9553,77.00,77.00,53,0,5.75,EG
342,Buraidah,26.3260,43.9750,77.00,77.00,29,0,4.61,SA
372,Sakakah,29.9697,40.2064,77.00,77.00,19,0,6.91,SA


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [114]:
#clean the data for readability 
hotel_df = narrow_df.loc[:, ["City", "Country", "Lat", "Lng"]]
#add empty column for hotel name. This will be filled in our next section
hotel_df["Hotel Name"] = ""

hotel_df

,City,Country,Lat,Lng,Hotel Name
4,Port Alfred,ZA,-33.5906,26.8910,
85,East London,ZA,-33.0153,27.9116,
106,Safut,JO,32.0336,35.8332,
139,Pisco,PE,-13.7000,-76.2167,
169,Mar del Plata,AR,-38.0023,-57.5575,
199,Lima,PE,-12.0432,-77.0282,
304,Karratha,AU,-20.7377,116.8463,
326,Alexandria,EG,31.2156,29.9553,
342,Buraidah,SA,26.3260,43.9750,
372,Sakakah,SA,29.9697,40.2064,


In [115]:
#base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
#search parameters
params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}
#use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    #add lat and lng in a f'string as a search parameter
    params["location"] = f"{lat},{lng}"

    #print results to make sure the api is working
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    #extract results
    results = response['results']
    
    #Save the hotel name to hotel_df data frame
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']
    except (KeyError, IndexError):
        print("Could not find result")
        print("-------------------------")

Retrieving Results for Index 4: Port Alfred.
Closest hotel in Port Alfred is MyPond Hotel.
Retrieving Results for Index 85: East London.
Closest hotel in East London is Garden Court East London.
Retrieving Results for Index 106: Safut.
Closest hotel in Safut is Saray Hotel Amman.
Retrieving Results for Index 139: Pisco.
Closest hotel in Pisco is Hostal Tambo Colorado.
Retrieving Results for Index 169: Mar del Plata.
Closest hotel in Mar del Plata is NH Gran Hotel Provincial.
Retrieving Results for Index 199: Lima.
Closest hotel in Lima is Hotel Diamond.
Retrieving Results for Index 304: Karratha.
Closest hotel in Karratha is Best Western Plus The Ranges Karratha.
Retrieving Results for Index 326: Alexandria.
Closest hotel in Alexandria is Hilton Alexandria Corniche.
Retrieving Results for Index 342: Buraidah.
Closest hotel in Buraidah is فندق تاورز HOTELTOWERS.
Retrieving Results for Index 372: Sakakah.
Closest hotel in Sakakah is Alnuzl Special Hotel.
Retrieving Results for Index 516:

In [116]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [117]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))